lecturesinparis

In [1]:
#libs
import json
from requests_html import HTMLSession

In [2]:
session = HTMLSession()

In [13]:
websites =  {# Institut Culturel Italien de Paris
        "url": "https://iicparigi.esteri.it/fr/gli_eventi/calendario/?date-init&date-end&searchText",
        "selectors": {
            "titles": ".card-title.big-heading.text-uppercase a",  # Targets the event title
            "descriptions": ".card-text",  # Targets the event description
            "dates": ".category-top .data",  # Targets the date
            "time": "",  # Leave empty if not available
            "locations": "",  # Leave empty as the location is static
            "links": ".card-title.big-heading.text-uppercase a"  # Targets the link to the event
        },
        "individual_page_selectors": {
        "descriptions": ".entry-content p",  # For the individual event page
        "dates": ".entry-meta .list-inline-item small",  # For the individual event page
        "time": ".entry-meta .list-inline-item small"  # For the individual event page
    },
        "static_location": "Institut Culturel Italien de Paris",  # Static location
        "pagination": "&page=2",  # Pagination format for this website
        }

all_events = [] # List to store all events

In [14]:
url = websites["url"]
selectors = websites["selectors"]
static_location = websites.get("static_location", "")

pagination = websites.get("pagination", None)
pagination_start = websites.get("pagination_start", 1)
page = pagination_start

# Fetch the page
try:
    response = session.get(url)
    response.raise_for_status()
except Exception as e:
    print(f"Failed to fetch {url}: {e}")

In [15]:
titles = response.html.find(selectors["titles"])
descriptions = response.html.find(selectors["descriptions"]) if selectors["descriptions"] else []
dates = response.html.find(selectors["dates"])
time = response.html.find(selectors["time"]) if selectors["time"] else []
locations = response.html.find(selectors["locations"]) if selectors["locations"] else []
links = response.html.find(selectors["links"])

# Iterate through the titles and construct event data
for i in range(len(titles)):
    event = {
        "title": titles[i].text if i < len(titles) else "",
        "description": descriptions[i].text if i < len(descriptions) else "",
        "date": dates[i].text if i < len(dates) else "",
        "time": time[i].text if i < len(time) else "",
        "location": locations[i].text if i < len(locations) else static_location,
        "link": links[i].attrs['href'] if i < len(links) else ""
    }
    all_events.append(event)
# Print the list in a more readable format
for event in all_events:
    print(f"Title: {event['title']}")
    print(f"Description: {event['description']}")
    print(f"Date: {event['date']}")
    print(f"Time: {event['time']}")
    print(f"Location: {event['location']}")
    print(f"Link: {event['link']}")
    print("-" * 50)  # Separator for better readability

Title: Exposition « Maria Callas. Portraits des Archives Publifoto Intesa Sanpaolo »
Description: Sous le commissariat de Aldo Grasso, l’exposition présente une sélection de 91 images, dont beaucoup sont inédites ou n’ont jamais été exposées, couvrant la période allant de 1954 à 1970 et privilégiant une typologie de clichés propres à une agence photo journalistique, qui dépeint Callas dans la vie quotidienne en dehors de la scène. Les […]
Date: jeu Avr 10 2025
mer Juin 04 2025
Time: 
Location: Institut Culturel Italien de Paris
Link: https://iicparigi.esteri.it/fr/gli_eventi/calendario/mostra-maria-callas-ritratti-dallarchivio-publifoto-intesa-sanpaolo/
--------------------------------------------------
Title: Exposition / Sospensioni de Renata Rampazzi
Description: Sospensioni de Renata Rampazzi15 avril – 30 mai 2025Institut Italien de Culture de Paris – Hôtel de GalliffetVernissage : 15 avril, 18h00 La peinture est un temps suspendu. Un geste qui s’arrête, mais qui pourrait se poursu

In [16]:
for event in all_events:
    # Check if any field is missing
    missing_fields = {
        "description": not event["description"],
        "date": not event["date"],
        "time": not event["time"]
    }

    print(missing_fields)

    # If any field is missing and the event has a link, fetch the individual page
    if event["link"] and any(missing_fields.values()):
        try:
            event_response = session.get(event["link"])
            event_response.raise_for_status()

            # Use individual page selectors if provided
            individual_selectors = websites.get("individual_page_selectors", {})

            # Fetch missing fields in a single request
            if missing_fields["description"] and "descriptions" in individual_selectors:
                description_element = event_response.html.find(individual_selectors["descriptions"], first=True)
                if description_element:
                    event["description"] = description_element.text

            if missing_fields["date"] and "dates" in individual_selectors:
                date_element = event_response.html.find(individual_selectors["dates"], first=True)
                if date_element:
                    event["date"] = date_element.text

            if missing_fields["time"] and "time" in individual_selectors:
                time_element = event_response.html.find(individual_selectors["time"], first=True)
                if time_element:
                    event["time"] = time_element.text

        except Exception as e:
            print(f"Failed to fetch additional info from {event['link']}: {e}")

    # No need to append the event back to all_events, as it is already being modified in place

# Print the updated events
for event in all_events:
    print(f"Title: {event['title']}")
    print(f"Description: {event['description']}")
    print(f"Date: {event['date']}")
    print(f"Time: {event['time']}")
    print(f"Location: {event['location']}")
    print(f"Link: {event['link']}")
    print("-" * 50)  # Separator for better readability

{'description': False, 'date': False, 'time': True}
{'description': False, 'date': False, 'time': True}
{'description': False, 'date': True, 'time': True}
{'description': False, 'date': True, 'time': True}
{'description': False, 'date': True, 'time': True}
{'description': False, 'date': True, 'time': True}
{'description': False, 'date': True, 'time': True}
{'description': False, 'date': True, 'time': True}
{'description': False, 'date': True, 'time': True}
{'description': False, 'date': True, 'time': True}
Title: Exposition « Maria Callas. Portraits des Archives Publifoto Intesa Sanpaolo »
Description: Sous le commissariat de Aldo Grasso, l’exposition présente une sélection de 91 images, dont beaucoup sont inédites ou n’ont jamais été exposées, couvrant la période allant de 1954 à 1970 et privilégiant une typologie de clichés propres à une agence photo journalistique, qui dépeint Callas dans la vie quotidienne en dehors de la scène. Les […]
Date: jeu Avr 10 2025
mer Juin 04 2025
Time: 